# Monte Carlo Simulation

In [1]:
from classes.MyDataLogger import MyDataLogger, MySubDataLogger
from modules.helper_functions_tsp import (find_problem_size,
                                          cost_fn_fact, 
                                          find_distances_array,
                                          convert_integer_to_binary_list,
                                          )
from itertools import count
import random


In [2]:
from modules.config import (RESULTS_DIR,
                            RESULTS_FILE, 
                            GRAPH_DIR,
                            ENCODING,
                            GRAY,
                            DECODING_FORMULATION
                            )

PRODUCTION = False   #don't print store anything if False

from pathlib import Path
import csv

Instantiate datalogger

In [3]:
datalogger = MyDataLogger()
sdl_list = []



In [4]:
LOCATIONS = [4, 5, 6, 7, 8, 9, 10, 11, 12]
#LOCATIONS = [4, 5]
ITERATIONS = 1

In [ ]:
#entered manually from show_results.ipynb
#maximum cache size for Circuits 1, 2, 3, 4, 6 by location
cache_size = {4 : 8_016,
              5 : 30_336,
              6 : 170_262,
              7 : 407_199,
              8 : 807_210,
              9 : 953_795,
             10 : 1_014_396,
             11 : 1_012_825,
             12 : 1_025_235
                }

Main loop

In [6]:
print(f'Running Monte Carlo simulations for {DECODING_FORMULATION} formulation with Gray = {GRAY}.')
for iterations in range(ITERATIONS):
    print(f'Iteration {iterations} of {ITERATIONS}.')
    for i in range(len(LOCATIONS)):
        sdl = MySubDataLogger(runid = datalogger.runid)
        sdl.locations = LOCATIONS[i]
        sdl.formulation = DECODING_FORMULATION
        sdl.gray = GRAY
        sdl.hot_start = False
        sdl.monte_carlo = True
        qubits = find_problem_size(sdl.locations, sdl.formulation)
        sdl.qubits = qubits
        print(f'There are {qubits} qubits needed for {sdl.locations} locations in the {sdl.formulation} formulation.')
        distance_array, best_dist = find_distances_array(sdl.locations, 
                                                        print_comments=False,
                                                        )
        sdl.best_dist = best_dist 
        cost_fn = cost_fn_fact(sdl.locations,
                                distance_array, 
                                sdl.gray, 
                                method=sdl.formulation,
                            )
        sdl.cache_max_size = cache_size[sdl.locations]
        lowest_to_date = 999_999 #too high
        highest = 2**sdl.qubits-1
        for j in range(sdl.cache_max_size):
            #if j >= highest_string:
            #    break
            integer = random.randint(1, highest)
            binary_string_list = convert_integer_to_binary_list(integer, 
                                                                sdl.qubits, 
                                                                gray = sdl.gray)
            cost = cost_fn(binary_string_list)
            if cost < lowest_to_date:
                lowest_to_date = cost
                iterations_found = j
        
        print(f' The lowest cost for {sdl.locations} locations is {lowest_to_date} which was found after {iterations_found} iterations out of {sdl.cache_max_size}.')
        sdl.best_dist_found = lowest_to_date
        sdl.update_cache_statistics(cost_fn)
        print(f'Cache hits: {sdl.cache_hits}, cache misses: {sdl.cache_misses}, total cache calls: {sdl.cache_hits + sdl.cache_misses}.')
        sdl.save_results_to_csv()

Running Monte Carlo simulations for original formulation with Gray = False.
Iteration 0 of 1.
SubDataLogger instantiated.  Run ID = 20250729-22-04-54 - 22-04-54
There are 3 qubits needed for 4 locations in the original formulation.
Reading distance data
 The lowest cost for 4 locations is 21.0 which was found after 0 iterations out of 8016.
Cache hits: 8009, cache misses: 7, total cache calls: 8016.
Saving data to results\results.csv
SubDataLogger instantiated.  Run ID = 20250729-22-04-54 - 22-04-54
There are 5 qubits needed for 5 locations in the original formulation.
Reading distance data
 The lowest cost for 5 locations is 19.0 which was found after 20 iterations out of 30336.
Cache hits: 30305, cache misses: 31, total cache calls: 30336.
Saving data to results\results.csv
SubDataLogger instantiated.  Run ID = 20250729-22-04-54 - 22-04-54
There are 8 qubits needed for 6 locations in the original formulation.
Reading distance data
 The lowest cost for 6 locations is 241.0 which was f